<a href="https://colab.research.google.com/github/airas-org/airas/blob/develop/notebook/airas_dev57_agents4science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --index-url https://test.pypi.org/simple/ \
            --extra-index-url https://pypi.org/simple \
            airas==0.0.11.dev57

In [ ]:
import logging

log_format = "[%(levelname)s] %(name)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=log_format, force=True)


In [ ]:
import os
from google.colab import userdata

# ==============================================================================
# SECRET KEY CONFIGURATION
# ==============================================================================
# INSTRUCTIONS:
# 1. Open the "Secrets" manager (key icon) in the Google Colab sidebar.
# 2. Add each secret name listed below (e.g., "OPENAI_API_KEY") and paste
#    its corresponding value.
# ==============================================================================


# --- Required API Keys---

# OpenAI API Key
# URL: https://platform.openai.com/settings/organization/api-keys
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# GitHub Personal Access Token (Fine-grained)
# URL: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
#
# Required settings for the token:
# - Resource owner: "auto-res2"
# - Permissions:    Grant full access to all scopes (Repositories, Workflows, etc.).
os.environ["GITHUB_PERSONAL_ACCESS_TOKEN"] = userdata.get("GITHUB_PERSONAL_ACCESS_TOKEN")


# --- Additional API Keys ---

os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
os.environ["DEVIN_API_KEY"] = userdata.get("DEVIN_API_KEY")
os.environ["QDRANT_API_KEY"] = userdata.get("QDRANT_API_KEY")
# os.environ["SEMANTIC_SCHOLAR_API_KEY"] = userdata.get("SEMANTIC_SCHOLAR_API_KEY")

In [ ]:
from airas.features import (
    AnalyticSubgraph,
    CheckExperimentalResultsSubgraph,
    CreateBibfileSubgraph,
    CreateBranchSubgraph,
    CreateCodeSubgraph,
    CreateCodeWithDevinSubgraph,
    CreateExperimentalDesignSubgraph,
    CreateMethodSubgraphV2,
    ExtractReferenceTitlesSubgraph,
    FixCodeSubgraph,
    FixCodeWithDevinSubgraph,
    GenerateQueriesSubgraph,
    GetPaperTitlesFromDBSubgraph,
    GitHubActionsExecutorSubgraph,
    GithubDownloadSubgraph,
    GithubUploadSubgraph,
    HtmlSubgraph,
    JudgeExperimentExecutionSubgraph,
    LatexSubgraph,
    PrepareRepositorySubgraph,
    ReadmeSubgraph,
    RetrieveCodeSubgraph,
    RetrievePaperContentSubgraph,
    ReviewPaperSubgraph,
    SummarizePaperSubgraph,
    WriterSubgraph,
)

In [ ]:
# ==============================================================================
# CONFIGURATION: DEFINE YOUR RESEARCH TASK
# ==============================================================================
# This is the primary cell for user input.
# Modify the variables below to set up your repository and research goal.
# ==============================================================================


# --- GitHub Repository Settings ---

# Set the repository owner.
github_owner = "auto-res2"

# Set the repository name.
repository_name = "your-reposiroty-name"

# Set the branch name.
# Each branch represents a distinct attempt or experiment for the research topic.
branch_name = "your-branch-name"


# --- Research Topic ---

# Describe your research topic here.
# Example: "I want to speed up the diffusion model."
research_topic = "your-research-topic"

In [ ]:
# ==============================================================================
# LLM SELECTION
# ==============================================================================
# This cell defines the list of all supported Large Language Models (LLMs).
# Any model in these lists can be used within any subgraph.
# ==============================================================================


# --- Supported Models ---

OPENAI_MODEL = [
    "o4-mini-2025-04-16",
    "o3-2025-04-16",
    "o3-mini-2025-01-31",
    "o1-pro-2025-03-19",
    "o1-2024-12-17",
    "gpt-5-2025-08-07",
    "gpt-5-mini-2025-08-07",
    "gpt-5-nano-2025-08-07",
    "gpt-4.1-2025-04-14",
    "gpt-4o-2024-11-20",
    "gpt-4o-mini-2024-07-18",
]

# If GEMINI_API_KEY is available
VERTEXAI_MODEL = [
    "gemini-2.5-pro",
    "gemini-2.5-flash",
    "gemini-2.5-flash-lite-preview-06-17",
    "gemini-2.0-flash-001",
    "gemini-2.0-flash-lite-001",
    "gemini-embedding-001",
]


In [ ]:
# ==============================================================================
# Subgraph Configuration
# ==============================================================================
# In this cell, we instantiate all the necessary subgraphs for the research
# workflow. They are grouped into logical phases for clarity:
# 1. Research and Information Gathering
# 2. Implementation and Experimentation
# 3. Writing and Publication

In [ ]:
# --- Phase 1: Research and Information Gathering ---


# Subgraph for generating search queries based on the research topic.
generate_queries = GenerateQueriesSubgraph(
    llm_mapping={
        "generate_queries": "o3-2025-04-16",
    },
    n_queries=5,
)


# Subgraph for retrieving paper titles from a local database.
get_paper_titles = GetPaperTitlesFromDBSubgraph(
    max_results_per_query=5, semantic_search=True
)


# Subgraph for retrieving the full content of papers.
retrieve_paper_content = RetrievePaperContentSubgraph(
    target_study_list_source="research_study_list",
    llm_mapping={
        "search_arxiv_id_from_title": "gpt-5-mini-2025-08-07",  # Only openAI models are available.
    },
    paper_provider="arxiv",
)

# Subgraph for summarizing the content of retrieved papers.
summarize_paper = SummarizePaperSubgraph(
    llm_mapping={"summarize_paper": "gemini-2.5-flash"}
)


# Subgraph for retrieving code related to the research papers.
retrieve_code = RetrieveCodeSubgraph(
    llm_mapping={
        "extract_github_url_from_text": "gemini-2.5-flash",
        "extract_experimental_info": "gemini-2.5-flash",
    }
)


# Subgraph for extracting reference titles from papers.
reference_extractor = ExtractReferenceTitlesSubgraph(
    llm_mapping={"extract_reference_titles": "gemini-2.5-flash"},
    paper_retrieval_limit=20,
)


# Subgraph for retrieving the content of reference papers.
retrieve_reference_paper_content = RetrievePaperContentSubgraph(
    target_study_list_source="reference_research_study_list",
    llm_mapping={
        "search_arxiv_id_from_title": "gpt-5-mini-2025-08-07",  # Only openAI models are available.
    },
    paper_provider="arxiv",
)

In [ ]:
# --- Phase 2: Implementation and Experimentation ---


# Subgraph for creating a new research method based on the findings.
create_method = CreateMethodSubgraphV2(
    llm_mapping={
        "generate_ide_and_research_summary": "o3-2025-04-16",
        "evaluate_novelty_and_significance": "o3-2025-04-16",
        "refine_idea_and_research_summary": "o3-2025-04-16",
        "search_arxiv_id_from_title": "gpt-5-mini-2025-08-07",  # Only openAI models are available.
    },
    refine_iterations=5,
)


# Subgraph for designing experiments to validate the new method.
create_experimental_design = CreateExperimentalDesignSubgraph(
    llm_mapping={
        "generate_experiment_strategy": "gpt-5-2025-08-07",
        "generate_experiment_specification": "gpt-5-2025-08-07",
        "generate_experiment_code": "gpt-5-2025-08-07",
    }
)


# Subgraph for generating code to implement the experimental design.
coder = CreateCodeSubgraph(
    llm_mapping={
        "generate_code_for_scripts": "gpt-5-2025-08-07",
    }
)


# Subgraph for executing code using GitHub Actions.
executor = GitHubActionsExecutorSubgraph(gpu_enabled=True)


# Subgraph for determining whether the experiment succeeded
judge_execution = JudgeExperimentExecutionSubgraph()


# Subgraph for fixing code errors.
fixer = FixCodeSubgraph(
    llm_mapping={
        "should_fix_code": "gpt-5-2025-08-07",
        "fix_code": "gpt-5-2025-08-07",
    }
)


# Subgraph for analyzing the results of the experiments.
analysis = AnalyticSubgraph(
    llm_mapping={
        "analytic_node": "gpt-5-2025-08-07",
    }
)

In [ ]:
# --- Phase 3: Writing and Publication ---


# Subgraph for creating a BibTeX file for citations.
create_bibfile = CreateBibfileSubgraph(
    llm_mapping={
        "filter_references": "gemini-2.5-flash",
    },
    latex_template_name="agents4science_2025",
    max_filtered_references=30,
)


# Subgraph for writing the final research paper.
writer = WriterSubgraph(
    llm_mapping={
        "write_paper": "gpt-5-2025-08-07",
        "refine_paper": "gpt-5-2025-08-07",
    },
    max_refinement_count=2,
)


# Subgraph for determining whether an experiment is running or exceeding the baseline
checker_results = CheckExperimentalResultsSubgraph()


# Subgraph for reviewing the generated paper against a set of criteria.
review = ReviewPaperSubgraph(
    llm_mapping={
        "review_paper": "gpt-5-2025-08-07",
    }
)


# Subgraph for generating a LaTeX version of the paper.
latex = LatexSubgraph(
    llm_mapping={
        "convert_to_latex": "gpt-5-2025-08-07",
        "is_execution_successful": "gpt-5-2025-08-07",
        "fix_latex_text": "gpt-5-2025-08-07",
    },
    latex_template_name="agents4science_2025",
    max_revision_count=3,
)


# Subgraph for generating an HTML version of the paper.
html = HtmlSubgraph(
    llm_mapping={
        "convert_to_html": "gpt-5-2025-08-07",
    }
)


# Subgraph for generating a README file for the project.
readme = ReadmeSubgraph()


# Subgraph for uploading the project to GitHub.
uploader = GithubUploadSubgraph()

In [ ]:
# ==============================================================================
# PREPARE THE GITHUB REPOSITORY
# ==============================================================================

from airas.types.github import GitHubRepositoryInfo


# Set up GitHub repository information and research topic.
state = {
    "github_repository_info": GitHubRepositoryInfo(
      github_owner=github_owner,
      repository_name=repository_name,
      branch_name=branch_name
    ),
    "research_topic": research_topic,
}

_ = PrepareRepositorySubgraph().run(state)

In [ ]:
# ==============================================================================
# EXECUTE THE RESEARCH WORKFLOW
# ==============================================================================

from tqdm import tqdm

subgraph_list = [
    generate_queries,
    get_paper_titles,
    retrieve_paper_content,
    summarize_paper,
    retrieve_code,
    create_method,
    create_experimental_design,
    coder,
    executor,
    judge_execution,
    fixer,
    analysis,
    reference_extractor,
    retrieve_reference_paper_content,
    create_bibfile,
    writer,
    checker_results,
    review,
    latex,
    html,
    readme,
]

def run_subgraphs(subgraph_list, state, max_fix_attempts=5):
    for subgraph in tqdm(subgraph_list, desc="Executing Research Workflow"):
        subgraph_name = subgraph.__class__.__name__
        print(f"--- Running Subgraph: {subgraph_name} ---")

        if isinstance(subgraph, (FixCodeSubgraph, AnalyticSubgraph)):
            continue

        elif isinstance(subgraph, JudgeExperimentExecutionSubgraph):
            fix_attempts = 0
            while fix_attempts < max_fix_attempts:
                state = judge_execution.run(state)
                if state.get("is_experiment_successful") is True:
                    state = analysis.run(state)
                    break
                else:
                    state = fixer.run(state)
                    state = executor.run(state)
                    fix_attempts += 1
            else:
                print(
                    f"!!! Max fix attempts ({max_fix_attempts}) reached for {state['research_topic']}. Moving on. !!!"
                )
                state = analysis.run(state)
        else:
            state = subgraph.run(state)

        _ = uploader.run(state)
        print(f"--- Finished Subgraph: {subgraph_name} ---\n")


# Execute the research workflow
_ = run_subgraphs(subgraph_list, state)

In [ ]:
# ==============================================================================
# OPTIONAL: CREATE A NEW BRANCH AND RESTART FROM A SUBGRAPH
# ==============================================================================
# Use this cell if you want to create a new branch and re-run the process
# from a specific point.
#
# This is useful for experimenting with a different approach midway through
# the workflow without altering the original branch.
# ==============================================================================


# --- CONFIGURATION ---

# Set the new branch name.
new_branch_name = "your-new-branch-name"

# Set the subgraph name where you want execution to resume.
# Example: "CreateMethodSubgraph"
restart_from_subgraph = "subgraph-name-to-restart-from"


# Create a new branch.
_ = CreateBranchSubgraph(
    new_branch_name = new_branch_name,
    restart_from_subgraph = restart_from_subgraph,
).run(state)

In [ ]:
# To restart the workflow from a specific subgraph (e.g., the one defined in
# the `restart_from_subgraph` variable), manually comment out the lines
# in this list that come before your desired starting point.

from airas.types.github import GitHubRepositoryInfo

# EXAMPLE: To restart from `create_method`, modify the list like this:

subgraph_list = [
    # generate_queries,
    # get_paper_titles,
    # retrieve_paper_content,
    # summarize_paper,
    # retrieve_code,
    create_method,
    create_experimental_design,
    coder,
    executor,
    judge_execution,
    fixer,
    analysis,
    reference_extractor,
    retrieve_reference_paper_content,
    create_bibfile,
    writer,
    checker_results,
    review,
    latex,
    html,
    readme,
]


# Re-initialize GitHub repository information for the new branch.
state = {
    "github_repository_info": GitHubRepositoryInfo(
      github_owner=github_owner,
      repository_name=repository_name,
      branch_name=new_branch_name
    ),
}
# Download the state from the new branch
state = GithubDownloadSubgraph().run(state)

# Re-execute the research workflow
_ = run_subgraphs(subgraph_list, state)